## Crawl Goodreads Book Pages 

In [11]:
import requests
import time

import pandas as pd

# The core dataset with book titles, ISBNs and Goodreads URLs
tekla_file = '1_Romance books_top 20 Goodreads 2011-2023 - Sheet1.csv'

# The output directory
html_dir = 'reviews/'

# Read the core dataset as a pandas DataFrame and show the Goodreads link column
source_df = pd.read_csv(tekla_file)[0:3]
source_df.head()

,URL_romance
0,https://www.goodreads.com/book/show/61718053-h...
1,https://www.goodreads.com/book/show/61326735-l...
2,https://www.goodreads.com/book/show/61918816-y...


Next, download and store the book pages for all URLs in the core dataset.

In [24]:
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 3.8 MB/s eta 0:00:00ta 0:00:01


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

# Define sleep time to prevent banning from server
def sleep():
    sleep_time = 10 + random.randint(0, 10) + random.random()
    time.sleep(sleep_time)

# List of URLs to scrape
urls = source_df.URL_romance.to_list()

data = []

for url in urls:
     try:
         # Send a GET request to the URL
        response = requests.get(url)

        # Create a BeautifulSoup object
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract metadata
        title = soup.find("h1", attrs={"data-testid":"bookTitle"}).text.strip()
        author = soup.find("span", attrs={"data-testid":"name"}).text.strip()
        average_rating = soup.find("div", attrs={"class":"RatingStatistics__rating"}).text.strip()
        ratings_count = soup.find("span", attrs={"data-testid":"ratingsCount"}).text.strip()
        reviews_count = soup.find("span", attrs={"data-testid":"reviewsCount"}).text.strip()
        description = soup.find("div", attrs={"data-testid":"description"}).text.strip()
        genres = [genre.text.strip() for genre in soup.find_all("span", attrs={"class":"BookPageMetadataSection__genreButton"})]
        pages_format = soup.find("p", attrs={"data-testid":"pagesFormat"}).text.strip()
        publication_info = soup.find("p", attrs={"data-testid":"publicationInfo"}).text.strip()
        #currently_reading_signal = soup.find("div", attrs={"data-testid":"currentlyReadingSignal"}).text.strip()
        #to_read_signal = soup.find("div", attrs={"data-testid":"toReadSignal"}).text.strip()
        #awards = [award.text for award in soup.select('div.DescListItem dd a')]
        #isbn = soup.find('dt', string='ISBN').find_next("div", attrs={"data-testid":"contentContainer"}).text.strip()
        ratings_histogram = [rating.text.strip() for rating in soup.find_all("div", attrs={"class":"RatingsHistogram__labelTotal"})]

        # Append the extracted data to the list
        data.append({
            "title": title,
            "author": author,
            "average_rating": average_rating,
            "ratings_count": ratings_count,
            "reviews_count": reviews_count,
            "description": description,
            "genres": genres,
            "pages_format": pages_format,
            "publication_info": publication_info,
            #"currently_reading_signal": currently_reading_signal,
            #"to_read_signal": to_read_signal,
            #"awards": awards,
            #"isbn": isbn,
            "ratings_histogram": ratings_histogram
        })
        
     except Exception as e:
        print(f"Error: {e}")
        print(f"Skipping URL: {url}")
        continue

        sleep()

# Create a dataframe from the list
df = pd.DataFrame(data)
df.head()

,title,author,average_rating,ratings_count,reviews_count,description,genres,pages_format,publication_info,ratings_histogram
0,Happy Place,Emily Henry,4.00,"917,038 ratings","110,010 reviews",Harriet and Wyn have been the perfect couple s...,"[Romance, Fiction, Contemporary, Audiobook, Co...","400 pages, Hardcover","First published April 25, 2023","[310,324 (33%), 361,439 (39%), 192,186 (20%), ..."
1,"Love, Theoretically",Ali Hazelwood,4.11,"416,471 ratings","58,046 reviews",The many lives of theoretical physicist Elsie ...,"[Romance, Contemporary, Contemporary Romance, ...","389 pages, Paperback","First published June 13, 2023","[153,411 (36%), 175,880 (42%), 71,585 (17%), 1..."
2,Yours Truly,Abby Jimenez,4.33,"358,447 ratings","48,314 reviews","A novel of terrible first impressions, hilario...","[Romance, Fiction, Contemporary, Audiobook, Co...","416 pages, Paperback","First published April 11, 2023","[171,389 (47%), 139,921 (39%), 40,530 (11%), 5..."


In [9]:
df.to_csv("test_books_metadata.csv")

# Scrape reviews 

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
from selenium.common.exceptions import NoSuchElementException

# !!! Adapt this to scrape reviews for a list of URLs
url = "https://www.goodreads.com/book/show/61718053/reviews"
url = "https://book.douban.com/subject/2567698/comments/"

# Set up Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

# Initialize the Chrome driver
driver = webdriver.Chrome(options=options)

# Open Goodreads website
driver.get(url)
proxy = {'https':'http://127.0.0.1:2802', 'http':'http://127.0.0.1:2802'}#国内得翻墙，加入了VPN代理的Http端口
# Click on the "Load more reviews" button until all reviews are loaded
# !!!! This part need to be fixed to make sure you load more reviews beyond the first 30 displayed on a page !!!
click_count = 0



#Goodreads
while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='loadMore']")))
        actions = ActionChains(driver)
        actions.move_to_element(load_more_button).perform()
        load_more_button.click()
        time.sleep(2)  # Wait for the next batch of reviews to load
        click_count += 1
        if click_count >= 10:
            break
    except:
        break  # If the "Load more reviews" button is not found, assume all reviews are loaded

#Anobii
# Get the HTML content
html = requests.get(url,proxies=proxy)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html.text, 'html.parser')
ht_test = html.text
page_count = 0
comment_InfoList = []
Booktitle = driver.find_element(By.CLASS_NAME,'unstyled-link').text
SavePath = 'Book_'+Booktitle + '.csv'
while page_count<10 :
    # 获取reviews-list中的所有activity-item-review项目
    reviews_list = driver.find_element(By.CLASS_NAME, 'reviews-list')
    reviews = reviews_list.find_elements(By.TAG_NAME, 'app-activity-item-review')

    for review in reviews:
        comment_content = review.find_element(By.CLASS_NAME, 'text_overflow').text
        try:
            comment_likes = review.find_element(By.CLASS_NAME, 'anchor').text
        except NoSuchElementException:
             comment_likes = 0
        comment_InfoList.append({
                "comment_content": comment_content,
                "comment_likes": comment_likes,
            })
    # 尝试找到翻页按钮并点击
    try:
        # 定位下一页按钮
        next_button = driver.find_element(By.XPATH, "//li[@class='page-item ng-star-inserted']/a[@aria-label='Next']")
        # 使用 JavaScript 来点击元素
        driver.execute_script("arguments[0].click();", next_button)
        page_count += 1
        driver.implicitly_wait(10)  # 等待新页面加载
    except NoSuchElementException:
        break  # 没有找到翻页按钮，结束循环
df = pd.DataFrame(comment_InfoList)
df.to_csv(SavePath,encoding='utf-8-sig',index=False)
driver.quit()

In [ ]:
#Goodreads改
# Get the HTML content
html = requests.get(url,proxies=proxy)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html.text, 'html.parser')
ht_test = html.text
click_count = 0
comment_InfoList = []
Booktitle = soup.find("a", attrs={"data-testid":"title"}).text.strip()
SavePath = 'Book_'+Booktitle + '.csv'
while click_count<20 :
    try:
        load_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='loadMore']")))
        actions = ActionChains(driver)
        actions.move_to_element(load_more_button).perform()
        load_more_button.click()
        time.sleep(5)  # Wait for the next batch of reviews to load
        click_count += 1
        if click_count >= 20:
            break
    except NoSuchElementException:
        break  # 没有找到翻页按钮，结束循环
    # 获取reviews-list中的所有activity-item-review项目
reviews_list = driver.find_element(By.CLASS_NAME, 'ReviewsList')
reviews = reviews_list.find_elements(By.CLASS_NAME, 'ReviewCard__content')
for review in reviews:
    comment_content = review.find_element(By.CLASS_NAME, 'ReviewText').text
    try:
        comment_rating = (review.find_element(By.CLASS_NAME,'ShelfStatus').find_element(By.TAG_NAME,'span').get_attribute('aria-label').split())[1]
    except:
        comment_rating=0
    try:
        comment_likes = (review.find_element(By.CLASS_NAME, 'SocialFooter').find_element(By.CLASS_NAME, 'Button__labelItem').text.split())[0]
    except:
        comment_likes=0

    comment_InfoList.append({
            "comment_content": comment_content,
            "comment_rating":comment_rating,
            "comment_likes": comment_likes,
        })
df = pd.DataFrame(comment_InfoList)
df.to_csv(SavePath,encoding='utf-8-sig',index=False)
driver.quit()

In [ ]:
#在需要登陆的网站需要先获取cookies
import json
# 配置第二个 Chrome 选项
options_2 = Options()
options_2.add_argument("--disable-blink-features=AutomationControlled")
driver2 = webdriver.Chrome(options=options_2)
try:
    # 打开小红书主页并手动登录
    driver2.get(url)
    time.sleep(60)  # 给用户足够的时间手动登录

    # 获取登录后的 cookie
    cookies = driver.get_cookies()

    # 打印当前工作目录
    print("Current working directory: ", os.getcwd())
    # 将 cookie 保存到文件
    with open("cookies.json", "w") as file:
        json.dump(cookies, file)
    print("Cookies saved successfully.")
finally:
    driver2.quit()

In [18]:
#豆瓣
options_2 = Options()
options_2.add_argument("--disable-blink-features=AutomationControlled")
driver2 = webdriver.Chrome(options=options_2)
try:
    # 打开并手动登录
    driver2.get(url)
    time.sleep(60)  # 给用户足够的时间手动登录
    driver2.refresh()
except:
    printf('err')

comment_InfoList = []
Booktitle = driver2.find_element(By.ID,'content').find_element(By.TAG_NAME,'h1').text
SavePath = 'Book_'+Booktitle + '_douban.csv'
rating_array=['很差','较差','还行','推荐','力荐']#评分数组，用于获取对应评分title对应的索引rating

while comment_InfoList.__len__()<810 :
    reviews_list = driver2.find_element(By.CLASS_NAME, 'comment-list')
    reviews = reviews_list.find_elements(By.CLASS_NAME, 'comment-item')
    for review in reviews:
        comment_content = review.find_element(By.CLASS_NAME, 'comment-content').text
        try:
            rate_s = review.find_element(By.CLASS_NAME,'comment-info').find_element(By.TAG_NAME,'span').get_attribute('title')
            for index,string in enumerate(rating_array):
                if string == rate_s:
                    comment_rating = index+1
                    break
        except:
            comment_rating=0
        try:
            comment_likes = review.find_element(By.CLASS_NAME, 'vote-count').text
        except:
            comment_likes=0

        comment_InfoList.append({
                "comment_content": comment_content,
                "comment_rating":comment_rating,
                "comment_likes": comment_likes,
            })
    try:
        # 定位下一页按钮
        page_buttons = driver2.find_element(By.ID,'paginator').find_elements(By.TAG_NAME, 'a')
        for button in page_buttons:
            if button.text == '后页 >':
                next_button=button
                break
        # 使用 JavaScript 来点击元素
        driver2.execute_script("arguments[0].click();", next_button)
        driver2.implicitly_wait(10)  # 等待新页面加载
    except NoSuchElementException:
        break  # 没有找到翻页按钮，结束循环
df = pd.DataFrame(comment_InfoList)
df.to_csv(SavePath,encoding='utf-8-sig',index=False)
driver2.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".comment-list"}
  (Session info: chrome=127.0.6533.120)


In [38]:
# Find all review containers and create a list to store the review texts

#goodreads 的 class
#review_texts = [review.text.strip() for review in soup.find_all("section", attrs={"class":"ReviewText__content"})]

#gr
test_sec = soup.find_all('app-activity-item-review',attrs={"class":"text_overflow"})
text_test = soup.find_all("section", attrs={"class":"text_overflow"})
review_texts = [review.text.strip() for review in soup.find_all("section", attrs={"class":"text_overflow"})]#goodreads 的 class

# Create a Pandas DataFrame from the review texts
df = pd.DataFrame({'review_text': review_texts})

# Save the DataFrame to a CSV file
df.to_csv('test_goodreads_reviews.csv' ,index=False)

# Close the Chrome driver
driver.quit()

In [14]:
# If you need to test with just one review and .find()
#dct = {'ReviewText_content': review_texts}
#dct = {k:[v] for k,v in dct.items()}
#df = pd.DataFrame(dct)

In [15]:
df.head()

,review_text
